In [2]:
from openai import OpenAI
import gradio as gr
import json

config=None
with open (".config.json",'r') as f:
    config=json.load(f)

In [11]:
text="if Bugis MRT Stn is on fire, provide affected station, lines and response suggestion"

client = OpenAI(api_key=config["OPENAI_API_KEY"])
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": 
        """
            You are an emergency response advice provider, and your clients are traffic authorities.\n
            User will describe current emergencies to you. In addition to answering users' questions, you will also provide emergency response suggestions.\n
            If your answer contains MRT Station Name, you must also provide its station code.\n
            If your answer contains MRT Line Name, you must also provide its line code.\n
            If your answer contains bus stop name, you must also provide its code.\n

            You will predict the number of passenger that will be effected. use per day to count.\n
            You will predict which types of emergency services the user needs to call in a separate paragraph, which strictly include: hospital, fire station, police station. However,if you think user don't need any of those service, your answer will not include this part.\n
            Your answer should be as concise and professional as possible and should not contain any non-specific advice.\n
            You don't need to explain why for your advice.\n
            No approximate values are allowed in your answer. If you cannot provide an exact number, provide a range bounded by the exact number.\n
        """
        },
        {"role": "user", "content": text}
    ]
)

print(completion.choices[0].message.content)

Affected Station:
- Bugis MRT Station (EW12/DT14)

Affected Lines:
- East West Line (EWL)
- Downtown Line (DTL)

Emergency Response Suggestion:
1. Evacuate all passengers from Bugis MRT Station (EW12/DT14).
2. Divert the trains to skip Bugis Station on both East West Line (EWL) and Downtown Line (DTL).
3. Alert nearby stations such as Lavender MRT Station (EW11) and City Hall MRT Station (EW13/NS25) to prepare for increased passenger load.
4. Deploy additional buses at affected bus stops, such as Bugis Stn Exit C (01119) and Opp Bugis Stn Exit C (01112), to reroute passengers.

Number of affected passengers: 10,000 to 15,000 per day

Please turn to Quick Query Tab to query route choice for deployment.

Emergency Services Needed: 
- Fire station
- Police station


In [12]:
text=completion.choices[0].message.content

In [14]:
client = OpenAI(api_key=config["OPENAI_API_KEY"])
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": 
        """
            You are a data preparation system. You serve Singapore's transport system\n
            Your input will be a traffic emergency response report.\n
            You will analyze which emergency services are suggested to be deployed to which destinations in this report\n
            If you think there is no emergency service included in this report, you need to strictly return:[[]]\n
            Otherwise, your answer must strictly be in following format:[["emergencyServiceName1","destination1"],["emergencyServiceName2","destination2"],...]
        """
        },
        {"role": "user", "content": text}
    ]
)

print(completion.choices[0].message.content)

[["Fire station","Bugis MRT Station (EW12/DT14)"],["Police station","Bugis MRT Station (EW12/DT14)"]]


In [29]:
client = OpenAI(api_key=config["OPENAI_API_KEY"])
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": 
        """
            You are a data preparation system. You serve Singapore's transport system\n
            Your input will be a traffic emergency response report.\n
            You already know that in Singapore, a bus can carry about 400 to 700 passengers a day.\n
            If the report mentions the number of passengers affected per day, you need to calculate how many additional buses at least and at most need to be deployed to cover alternative travel.\n
            If you think there is no number of passengers affected included in the report, your will strictly return:[0,0]\n
            Otherwise, your answer will strictly be in following format:[leastNumber,mostNumber]
        """
        },
        {"role": "user", "content": text}
    ]
)

print(completion.choices[0].message.content)

[25, 38]


In [9]:
text="""
Affected Stations:\n
City Hall MRT Station (NS25/EW13): 13,200 passengers/hour\n
Raffles Place MRT Station (NS26/EW14): 12,000 passengers/hour\n
Clarke Quay MRT Station (NE5): 8,000 passengers/hour\n
Esplanade MRT Station (CC3): 7,000 passengers/hour\n
Deployment of Emergency Services:\n
Fire station to City Hall MRT Station (NS25/EW13)"""

client = OpenAI(api_key=config["OPENAI_API_KEY"])
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": 
        """
            You are a data analyst serving for Singaporean transport system. You calculate and summarize data\n
            The input is a emergency response report including several hourly numbers of passenger affected in several train stations and/or bus stops respectively.\n
            You will devide those numbers by 102, then match each devided number with corresponding station or stop name.\n
            You will list those devided numbers in following format: 1. name1 need devidedNumber1 extra bus trips per hour\n
            You may not show calculations.
        """
        },
        {"role": "user", "content": text}
    ]
)

In [10]:
print(completion.choices[0].message.content)

1. City Hall MRT Station (NS25/EW13) needs 129 extra bus trips per hour
2. Raffles Place MRT Station (NS26/EW14) needs 118 extra bus trips per hour
3. Clarke Quay MRT Station (NE5) needs 78 extra bus trips per hour
4. Esplanade MRT Station (CC3) needs 69 extra bus trips per hour
